In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# want to Implemement ADAMW as my optimizer...

In [ ]:

# Chopped and changed class
class FFN_change:
    def __init__(self, input_data, output_data, hidden_layers, activation_functions, epochs, learning_rate, batch_size):
        self.input_data = input_data  # Make sure this is a numpy array
        self.output_data = output_data
        self.hidden_layers = hidden_layers
        self.activation_functions = activation_functions
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.weights_dictionary = {}
        self.activations_dictionary = {}
        self.initialize()
##### an improvement could be if you have a method that allows you to input new data to further train your data
    def initialize(self):
        last_layer = self.input_data.shape[0]
        for i in range(len(self.hidden_layers)):
            self.weights_dictionary[f'w{i}'] = 0.1 * np.random.randn(self.hidden_layers[i], last_layer)
            self.weights_dictionary[f'b{i}'] = np.zeros((self.hidden_layers[i], 1))
            last_layer = self.hidden_layers[i]
        self.weights_dictionary['last_weight'] = 0.1 * np.random.randn(self.output_data.shape[0], last_layer)
        self.weights_dictionary['last_bias'] = np.zeros((self.output_data.shape[0], 1))
        
    def forward(self, input_batch): 
        for i in range(len(self.hidden_layers)):
            z = np.dot(self.weights_dictionary[f'w{i}'], input_batch) + self.weights_dictionary[f'b{i}']
            a = self.activation(z, self.activation_functions[i])
            self.activations_dictionary[f'z{i + 1}'] = z
            self.activations_dictionary[f'a{i + 1}'] = a
            input_batch = a  
        a = np.dot(self.weights_dictionary['last_weight'], input_batch) + self.weights_dictionary["last_bias"]
        self.activations_dictionary['activation_output'] = a

    def activation(self, a, activation_function):
        if activation_function[0] == "relu":
            return np.maximum(0, a)
        elif activation_function[0] == "sigmoid":
            return 1 / (1 + np.exp(-a))
        elif activation_function[0] == "tanh":
            return np.tanh(a)
        elif activation_function[0] == "lrelu":
            return np.maximum(activation_function[1] * a, a) 
        else:
            raise Exception("Invalid activation function")

    def activation_derivative(self, a, activation_function):
        if activation_function[0] == 'relu':
            return np.where(a > 0, 1, 0)
        elif activation_function[0] == 'sigmoid':
            sig = self.activation(a, ('sigmoid', 0))
            return sig * (1 - sig)
        elif activation_function[0] == 'tanh':
            return 1 - np.tanh(a) ** 2
        elif activation_function[0] == 'lrelu':
            dx = np.ones_like(a)
            dx[a <= 0] = activation_function[1]
            return dx
        else:
            raise Exception("Invalid activation function derivative")

    def backward(self, dvalues, input_batch):
        i = len(self.hidden_layers)
        self.weights_dictionary["dweights last_weight"] = np.dot(dvalues, self.activations_dictionary[f'a{i}'].T)
        self.weights_dictionary["dbiases last_bias"] = np.sum(dvalues, axis=1, keepdims=True)
        dinputs = np.dot(self.weights_dictionary["last_weight"].T, dvalues)
        
        for i in range(len(self.hidden_layers) - 1, -1, -1):
            dinputs *= self.activation_derivative(self.activations_dictionary[f'z{i + 1}'], self.activation_functions[i])
            self.weights_dictionary[f'dweights{i}'] = np.dot(dinputs, self.activations_dictionary[f'a{i}'].T if i > 0 else input_batch.T)
            self.weights_dictionary[f'dbiases{i}'] = np.sum(dinputs, axis=1, keepdims=True)
            if i > 0:
                dinputs = np.dot(self.weights_dictionary[f'w{i}'].T, dinputs)

    def compute_loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2) 

    def loss_backwards(self, predictions, targets):
        return 2 * (predictions - targets) / targets.shape[1] 

    def train(self):
        epoch_data = []
        loss_history = []
        for epoch in range(self.epochs):
            epoch_loss = 0
            batch_count = 0
            
            for start in range(0, self.input_data.shape[1], self.batch_size):
                end = min(start + self.batch_size, self.input_data.shape[1])
                input_batch = self.input_data[:, start:end]
                output_batch = self.output_data[:, start:end]
                
                self.forward(input_batch)
                loss = self.compute_loss(self.activations_dictionary["activation_output"], output_batch)
                dvalues = self.loss_backwards(self.activations_dictionary["activation_output"], output_batch)
                self.backward(dvalues, input_batch)
                self.update()
                
                epoch_loss += loss
                batch_count += 1
            
            epoch_loss /= batch_count
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {epoch_loss}")
                epoch_data.append(epoch)
                loss_history.append(epoch_loss)

        plt.plot(epoch_data, loss_history)
        plt.xlabel("Epoch Number")
        plt.ylabel("Loss")
        plt.title("Epoch Number vs Loss")
        plt.show()
        
    def update(self):
        for i in range(len(self.hidden_layers)):
            self.weights_dictionary[f'w{i}'] -= self.learning_rate * self.weights_dictionary[f'dweights{i}']
            self.weights_dictionary[f'b{i}'] -= self.learning_rate * self.weights_dictionary[f'dbiases{i}']
        self.weights_dictionary["last_weight"] -= self.learning_rate * self.weights_dictionary["dweights last_weight"]
        self.weights_dictionary["last_bias"] -= self.learning_rate * self.weights_dictionary["dbiases last_bias"]


    def predict(self, input_data, output_data):
        self.forward(input_data)
        loss = self.compute_loss(self.activations_dictionary["activation_output"], output_data)
        return loss, self.activations_dictionary["activation_output"]

    def forecast(self, input):
        self.forward(input)
        return self.activations_dictionary["activation_output"]
    

    